Ref: [Andy Balaam - Python Async basics video (100 million HTTP requests](https://www.youtube.com/watch?v=Mj-Pyg4gsPs)

In [ ]:
# Code from Andy Balaam - https://www.youtube.com/watch?v=Mj-Pyg4gsPs

# Ewald's fix for asyncio not working in Jupyter
# (Ref: https://github.com/erdewit/nest_asyncio)
import nest_asyncio
nest_asyncio.apply()

import asyncio
async def mycoro(number):
    print ("Starting %d" % number)  # Runs now
    await asyncio.sleep(1)
    print("Finishing %d" % number) # Runs later
    return str(number)

# c = mycoro(3)
# task = asyncio.create_task(c)

# task = asyncio.ensure_future(mycoro(1))
# loop = asyncio.get_event_loop()
# loop.run_until_complete(task)
# # loop.close() # to be suppressed in Jupyter!

asyncio.run(mycoro(1))

many = asyncio.gather(
        mycoro(1),
        mycoro(2),
        mycoro(3)
)

asyncio.run(many)

In [ ]:
# Code from Andy Balaam - https://www.youtube.com/watch?v=Mj-Pyg4gsPs

# Ewald's fix for asyncio not working in Jupyter
# (Ref: https://github.com/erdewit/nest_asyncio)
# import nest_asyncio
# nest_asyncio.apply()

import asyncio
async def mycoro(number):
    print ("Starting %d" % number)  # Runs now
    await asyncio.sleep(1)
    print("Finishing %d" % number) # Runs later
    return str(number)

# c = mycoro(3)
# task = asyncio.create_task(c)

# task = asyncio.ensure_future(mycoro(1))

# # loop.close() # to be suppressed in Jupyter!

# asyncio.run(mycoro(1))

# with asyncio.wait()
# many = asyncio.wait([
        # mycoro(1),
        # mycoro(2),
        # mycoro(3)]
# )

# with asynicio.gather(), it does not work
many = asyncio.gather(
        mycoro(1),
        mycoro(2),
        mycoro(3)
)

# loop = asyncio.get_event_loop()
# loop.run_until_complete(many)

asyncio.run(many)

In [ ]:
import asyncio
async def f2():
    print ("start f2")
    await asyncio.sleep(1)
    print("stop f2")

async def f1():
    print("start f1")
    await f2()
    print ("stop f1")
    
asyncio.run(f1())

In [1]:
from aiohttp import ClientSession, TCPConnector
import asyncio
from itertools import islice
import sys

# import nest_asyncio
# nest_asyncio.apply()

async def fetch(url):
    async with ClientSession() as s, s.get(url) as res:
        ret = await res.read()
        print(ret)
        return ret
    
# asyncio.run(fetch("http://example.com"))

def limited_as_completed(coros, limit):
    futures = [
        asyncio.ensure_future(c)
        for c in islice(coros, 0, limit)
    ]
    async def first_to_finish():
        while True:
            await asyncio.sleep(0)
            for f in futures:
                if f.done():
                    futures.remove(f)
                    try:
                        newf = next(coros)
                        futures.append(
                            asyncio.ensure_future(newf))
                    except StopIteration as e:
                        pass
                    return f.result()
    while len(futures) > 0:
        yield first_to_finish()

# code using asyncio.as_completed() for intermittent gather
async def print_when_done(tasks):
    for res in limited_as_completed(tasks, 100):
        print(await res)
        
coros = [
    fetch("http://example.com")
    for i in range(1000)
]

asyncio.run(print_when_done(coros))